## 14.1 텐서플로의 주요 특징

## 14.2 텐서플로의 계산 그래프: 텐서플로 v2로 이전

* 즉시 실행, 동적 계산 그래프

### 14.2.6 함수 데코레이터로 계산 성능 높이기

`@tf.function`: 정적 타입과 크기가 필요하지만 이를 동적 타이핑으로 처리할 수 있음

## 14.3 모델 파라미터를 저장하고 업데이트하기 위한 텐서플로 변수 객체

In [1]:
# 랜덤 초기화의 일종 고전적인 랜덤 초기화 방법: 글로럿 초기화 (GlorotNormal)

import tensorflow as tf

tf.random.set_seed(1)
init = tf.keras.initializers.GlorotNormal()
tf.print(init(shape=(3, )))

[-0.722795904 1.01456821 0.251808226]


2022-04-25 07:06:41.822082: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
v = tf.Variable(init(shape=(2, 3)))
tf.print(v)

[[0.28982234 -0.782292783 -0.0453658961]
 [0.960991383 -0.120003454 0.708528221]]
